# Python 2 type annotation

In [2]:
color_love_dict = {
    'Red': 0,
    'Blue': 4,
    'Purple': 90001
    }  # type: dict
print(color_love_dict)

{'Red': 0, 'Blue': 4, 'Purple': 90001}


# Python 3 type annotation

In [5]:
from typing import Dict
color_love_dict: Dict = {
    'Red': 0,
    'Blue': 4,
    'Purple': 90001
    }
print(color_love_dict)

{'Red': 0, 'Blue': 4, 'Purple': 90001}


## The **RIGHT** type annotation is to not annotate at all

In [6]:
color_love_dict = {
    'Red': 0,
    'Blue': 4,
    'Purple': 90001
    }
print(color_love_dict)

{'Red': 0, 'Blue': 4, 'Purple': 90001}


### Setup

In [12]:
import requests
from mypy import api


def get_color_love():
    color_love_url = "http://my-json-server.typicode.com/jochman/colors/love"
    if res := requests.get(color_love_url, verify=False).json():
        return res 
    return None

def get_color_hate():
    color_love_url = "http://my-json-server.typicode.com/jochman/colors/love"
    if res := requests.get(color_love_url, verify=False).json():
        return res 
    return None

# Where problem starts


In [5]:
color_love_dict = get_color_love()

print(color_love_dict.items())

dict_items([('Red', 0), ('Blue', 4), ('Purple', 90001)])


In [11]:
print(api.run(['./no_annotation.py'])[0])

no_annotation.py:3: error: Item "None" of "Optional[Dict[Any, Any]]" has no attribute "keys"
Found 1 error in 1 file (checked 1 source file)



## Common case

In [9]:
color_love_dict = get_color_love()  # type: ignore

print(color_love_dict.items())

b''


In [ ]:
print(api.run(['./typeignore.py'])[0])